In [2]:
import os
import json

def convert_coco_to_yolo(coco_json_path, output_dir, image_dir):
    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Create output directory for YOLO labels
    os.makedirs(output_dir, exist_ok=True)

    # os.makedirs(output_dir + "/images", exist_ok=True)
    
    # Get categories and map them to YOLO class IDs
    categories = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}
    
    # Create a dictionary to store annotations per image
    annotations = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        bbox = ann['bbox']
        category_id = categories[ann['category_id']]
        
        # Convert COCO bbox to YOLO bbox
        x_center = bbox[0] + bbox[2] / 2.0
        y_center = bbox[1] + bbox[3] / 2.0
        width = bbox[2]
        height = bbox[3]
        
        # Normalize by image width and height
        image_info = next((img for img in coco_data['images'] if img['id'] == image_id), None)
        if image_info is None:
            print(f"Image ID {image_id} not found in 'images' section of COCO JSON")
            continue
        
        img_width = image_info['width']
        img_height = image_info['height']
        
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        # Add to annotations dict
        if image_id not in annotations:
            annotations[image_id] = []
        annotations[image_id].append(f"{category_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    # Write YOLO .txt files
    for image in coco_data['images']:
        image_id = image['id']
        file_name = os.path.splitext(image['file_name'])[0]  # Get file name without extension
        label_path = os.path.join(output_dir, f"{file_name}.txt")
        
        with open(label_path, 'w') as label_file:
            if image_id in annotations:
                label_file.write("\n".join(annotations[image_id]))
    
    print(f"Conversion completed! YOLO labels saved to {output_dir}")

# Example usage
coco_json_path = "C:\\Users\\tfgmo\\OneDrive - Virginia Tech\\Mahjong CV\\augmented_dataset_splited\\test\\annotations.json"  # Path to your COCO JSON file
output_dir = "C:\\Users\\tfgmo\\OneDrive - Virginia Tech\\Mahjong CV\\augmented_dataset_splited\\test\\"        # Directory to save YOLO labels
image_dir = "C:\\Users\\tfgmo\\OneDrive - Virginia Tech\\Mahjong CV\\augmented_dataset_splited\\test\\"                # Directory containing the images

convert_coco_to_yolo(coco_json_path, output_dir, image_dir)


Conversion completed! YOLO labels saved to C:\Users\tfgmo\OneDrive - Virginia Tech\Mahjong CV\augmented_dataset_splited\test\
